# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
drive = 'D'
ods_fold = f'{drive}:/data/big2/ODS'
analysis_fold = 'analysis'

# load

In [4]:
# about 8m 7.8s
car = pd.read_csv(os.path.join(ods_fold, '[ODS]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), low_memory=False)

In [5]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25404385 entries, 0 to 25404384
Data columns (total 59 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          object 
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          ob

In [6]:
car.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

# 분석

In [7]:
car['등급'].value_counts(dropna=False)

2    11207677
3     8759393
1     2771380
4     1511211
5     1154724
Name: 등급, dtype: int64

In [8]:
car['연료'].value_counts(dropna=False)

휘발유            12101949
경유              9860384
LPG(액화석유가스)     1896593
휘발유 하이브리드       1071363
전기               366804
경유 하이브리드          34191
CNG(압축천연가스)       30311
수소                28102
LPG 하이브리드         14350
CNG 하이브리드           304
LNG(액화천연가스)          25
무동력                   7
알코올                   2
Name: 연료, dtype: int64

## 전체 차량 현황

In [17]:
stat_car = car.groupby(['대기관리권역', '시도', '대기관리권역YN'], dropna=False)['차대번호'].count().unstack()
stat_car

대기관리권역YN          N          Y
대기관리권역 시도                     
그 외    강원 846993.00        NaN
       제주 695572.00        NaN
남부권    광주       NaN  711165.00
       전남 616235.00  591351.00
동남권    경남 537576.00 1342341.00
       경북 582214.00  934617.00
       대구       NaN 1228482.00
       부산       NaN 1490113.00
       울산       NaN  585909.00
수도권    경기 128583.00 6207209.00
       서울       NaN 3209359.00
       인천   9072.00 1707894.00
중부권    대전       NaN  705277.00
       세종       NaN  191861.00
       전북 344562.00  628490.00
       충남  31389.00 1168268.00
       충북 120426.00  789427.00

In [19]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.05.04_PM.04.13.06'

In [20]:
with pd.ExcelWriter(os.path.join(analysis_fold, f'통계_전체_대기관리권역별{today_date}.xlsx')) as writer:
    stat_car.to_excel(writer, sheet_name='전체')

## 전기차와 수소

In [9]:
df = car.loc[(car['연료'] == '전기') | (car['연료'] == '수소')].reset_index(drop=True)
df.shape

(394906, 59)

In [10]:
stat = df.groupby(['대기관리권역', '시도', '대기관리권역YN'], dropna=False)['차대번호'].count().unstack()
stat

대기관리권역YN         N        Y
대기관리권역 시도                  
그 외    강원 15487.00      NaN
       제주 31654.00      NaN
남부권    광주      NaN  9552.00
       전남  6349.00  8904.00
동남권    경남  5758.00 17552.00
       경북  7381.00 10716.00
       대구      NaN 23556.00
       부산      NaN 22396.00
       울산      NaN  7202.00
수도권    경기  1861.00 75599.00
       서울      NaN 60653.00
       인천    51.00 25263.00
중부권    대전      NaN 14624.00
       세종      NaN  3025.00
       전북  5190.00  8912.00
       충남   432.00 16747.00
       충북  1879.00 14163.00

## 전기차

In [11]:
ev = df.loc[df['연료'] == '전기'].reset_index(drop=True)
ev.shape

(366804, 59)

In [12]:
ev_stat = ev.groupby(['대기관리권역', '시도', '대기관리권역YN'], dropna=False)['차대번호'].count().unstack()
ev_stat

대기관리권역YN         N        Y
대기관리권역 시도                  
그 외    강원 12991.00      NaN
       제주 31649.00      NaN
남부권    광주      NaN  8458.00
       전남  6265.00  8288.00
동남권    경남  5552.00 15379.00
       경북  7314.00 10568.00
       대구      NaN 23030.00
       부산      NaN 20517.00
       울산      NaN  4764.00
수도권    경기  1835.00 69655.00
       서울      NaN 57819.00
       인천    50.00 23703.00
중부권    대전      NaN 13417.00
       세종      NaN  2783.00
       전북  4770.00  7642.00
       충남   424.00 15571.00
       충북  1815.00 12545.00

## 하이브리드

In [13]:
h2 = df.loc[df['연료'] == '수소'].reset_index(drop=True)
h2.shape

(28102, 59)

In [14]:
h2_stat = h2.groupby(['대기관리권역', '시도', '대기관리권역YN'], dropna=False)['차대번호'].count().unstack()
h2_stat

대기관리권역YN        N       Y
대기관리권역 시도                
그 외    강원 2496.00     NaN
       제주    5.00     NaN
남부권    광주     NaN 1094.00
       전남   84.00  616.00
동남권    경남  206.00 2173.00
       경북   67.00  148.00
       대구     NaN  526.00
       부산     NaN 1879.00
       울산     NaN 2438.00
수도권    경기   26.00 5944.00
       서울     NaN 2834.00
       인천    1.00 1560.00
중부권    대전     NaN 1207.00
       세종     NaN  242.00
       전북  420.00 1270.00
       충남    8.00 1176.00
       충북   64.00 1618.00

## 출력

In [15]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.05.04_PM.03.59.12'

In [16]:
with pd.ExcelWriter(os.path.join(analysis_fold, f'통계_전기차_수소_대기관리권역별{today_date}.xlsx')) as writer:
    stat.to_excel(writer, sheet_name='전기_수소')
    ev_stat.to_excel(writer, sheet_name='전기차')
    h2_stat.to_excel(writer, sheet_name='수소')

# 코드 마지막